### Import Necessary Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta

### Constant

In [2]:
train_start_dt = datetime(2017, 7, 1, 0, 0, 0)
train_end_dt = datetime(2020, 6, 30, 23, 0, 0)

test_start_dt = datetime(2020, 7, 1, 0, 0, 0)
test_end_dt = datetime(2021, 7, 1, 23, 0, 0)

LOCS = ["BKK", "Chiangmai", "Khonkaen", "Rayong", "Saraburi", "Surat"]
REGIONS = ['Thailand', 'Myanmar', 'Cambodia', 'Lao_PDR', 'P_Malaysia']

base_scraped_data_path = './scraped_data'
base_processed_data_path = './processed_data'
base_dataset_path = './datasci_dataset_2022'

## General Functions

In [3]:
def read_scraped_data(base_scraped_data_path, mode):
    '''
    This function will read the scraped data (csv) file with the given mode, and return a pandas DataFrame
    [Input]
        - base_scraped_data_path: string containing folder of scraped data path
        - mode: "temp" / "wind" / "hotspot"
    [Output]
        - df: pandas DataFrame
    '''
    
    df = pd.read_csv(f'{base_scraped_data_path}/{mode}_scraped_data.csv')
    
    if (mode == "wind" or mode == "temp"):
        df['date_time'] = pd.to_datetime(df['date_time'], format = '%Y-%m-%d %H:%M:%S')
        df.sort_values(by = 'date_time', inplace = True)
    elif (mode == "hotspot"):
        df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
        df.sort_values(by = 'date', inplace = True)
        
    df.drop_duplicates(inplace = True)
    return df

def read_wind_or_temp_train_data(base_dataset_path, province, mode):
    '''
    This function will read train data (csv) file with given province, mode, and then return a pandas DataFrame
    [Input]
        - base_dataset_path: string containing folder of the dataset path
        - province: ["BKK", "Chiangmai", "Khonkaen", "Rayong", "Saraburi", "Surat"]
        - mode: "wind" or "temp"
    [Output]
        - df: pandas DataFrame
    '''
    
    path = f'{base_dataset_path}/{province}/train'
    
    if (mode == "wind"):
        path = f'{path}/{province.lower()}_weather_wind.csv'
        df = pd.read_csv(path)
        
        if (province in ["BKK", "Khonkaen", "Rayong"]):
            df['date_time'] = pd.to_datetime(df['date_time'], format = '%Y-%m-%d %H:%M:%S')
        elif (province in ["Chiangmai", "Surat"]):
            df['date_time'] = pd.to_datetime(df['date_time'], format = '%m/%d/%Y %H:%M')
        elif (province in ["Saraburi"]):
            df['date_time'] = pd.to_datetime(df['date_time'], format = '%d/%m/%Y %H:%M')
        
        df.rename(columns = {'wind speed': f'wind_speed_{province}', 'wind dir': f'wind_dir_{province}'}, inplace = True)
    
    elif (mode == "temp"):
        path = f'{path}/{province.lower()}_temp_surface.csv'
        df = pd.read_csv(path)
        
        if (province in ["BKK", "Chiangmai", "Rayong", "Saraburi", "Surat"]):
            df['date_time'] = pd.to_datetime(df['date_time'], format = '%m/%d/%Y %H:%M')
        elif (province in ["Khonkaen"]):
            df['date_time'] = pd.to_datetime(df['date_time'], format = '%Y-%m-%d %H:%M:%S')
        
        df.rename(columns = {'temp': f'temp_{province}'}, inplace = True)

    df.sort_values(by = 'date_time', inplace = True)
    df.drop_duplicates(inplace = True)
    return df

def get_current_dt():
    current_dt = datetime.now()
    return current_dt.strftime('%Y-%m-%d %H:%M:%S')


def save_result_from_processing(df, mode):
    '''
    This function receives dataframe containing datas and save into folder ./processed_data with given mode
    [Input]
        df - pandas DataFrame
        mode - "wind", "temp", "hotspot"
    '''
    
    #check if path exists (if not create it)
    base_path = './processed_data'
    if not os.path.exists(base_path):
        os.mkdir(base_path)
        
    #drop duplicates if exists
    if (df.duplicated().sum() != 0):
        print(f'Dropping {df.duplicated().sum()} duplicated rows')
        df.drop_duplicates(inplace = True)
    
    #save dataframe
    path = f'./{base_path}/{mode}_processed_data.csv'
    df.to_csv(path, index = False)

## Process Hotspot Data

1. Load the daily hotspot data scraped from the internet
2. Note that there are some missing values

   [For Train Duration (2017-07-01 to 2020-06-30)]
       at any date x we will perform linear interpolation between date x-1 and date x+1
   [For Test Duration (2020-07-01 to 2021-07-01)]
       to prevent using the future data
       at any date x we will use data from the date x-1 only
3. Change resoulution from daily to 6H (0:00, 6:00, 12:00, 18:00)
4. [For Train Duration (2017-07-01 to 2020-06-30)]
       at any date x
           - 0:00 and 6:00  we will use the hotspot counts at "night" time of date x divided by 12 (hours)
           - 12:00 and 18:00 we use the hotspot counts at "day" time of date x divided by 12 (hours)
        
   [For Test Duration (2020-07-01 to 2021-07-01)]
       to prevent using the future data
       at any date x
           - 0:00 and 6:00  we will use the hotspot counts at "night" time of date "x-1" divided by 12 (hours)
           - 12:00 and 18:00 we will use the hotspot counts at "day" time of date "x-1" divided by 12 (hours)

Note: after divided by 12 we will round it up

In [4]:
def process_hotspot_data(df_hotspot_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, REGIONS):
    # column ==================================================================================
    day_cols = df_hotspot_scraped.columns[[i for i in range (1, len(df_hotspot_scraped.columns), 2)]]
    night_cols = df_hotspot_scraped.columns[[i for i in range (2, len(df_hotspot_scraped.columns), 2)]]
    column_names = [f'hotspot_{r.lower()}' for r in REGIONS]  
    
    #day_cols = ['hotspot_thailand_day', 'hotspot_myanmar_day', 'hotspot_cambodia_day', 'hotspot_lao_pdr_day', 'hotspot_p_malaysia_day']
    #night_cols = ['hotspot_thailand_night', 'hotspot_myanmar_night', 'hotspot_cambodia_night', 'hotspot_lao_pdr_night', 'hotspot_p_malaysia_night']
    #column_names = ['date_time', 'hotspot_thailand', 'hotspot_myanmar', 'hotspot_cambodia', 'hotspot_lao_pdr', 'hotspot_p_malaysia']
    # =========================================================================================
    
    # separate df_hotspot into df_hotspot_train, df_hotspot_test
    df_hotspot_train = df_hotspot_scraped[(train_start_dt <= df_hotspot_scraped['date']) & (df_hotspot_scraped['date'] <= train_end_dt)].copy()
    df_hotspot_test = df_hotspot_scraped[(test_start_dt <= df_hotspot_scraped['date']) & (df_hotspot_scraped['date'] <= test_end_dt)].copy()
    
    # for train duration (perform normal interpolation)
    df_hotspot_train.interpolate(inplace = True)
    # for test duration (fill the missing values with previous value only - !!!prevent using future data)
    df_hotspot_test.interpolate(method = 'pad', limit_direction = 'forward', inplace = True)
    
    #change resolution from daily to 6H (train)
    train_daterange = pd.date_range(start = train_start_dt, end = train_end_dt, freq = '6H')
    train_hotspot_data = []
    for dt in train_daterange:
        current_date = dt.date()
        hour = dt.time().hour
        
        data = pd.Series(data = [dt], index = ['date_time'])
        
        night_data = df_hotspot_train.loc[df_hotspot_train["date"] == pd.to_datetime(str(current_date), format = '%Y-%m-%d'), night_cols].iloc[0].to_numpy()
        day_data = df_hotspot_train.loc[df_hotspot_train["date"] == pd.to_datetime(str(current_date), format = '%Y-%m-%d'), day_cols].iloc[0].to_numpy()
        
        if (hour == 0 or hour == 6):
            hotspot_data = night_data.copy()
        elif (hour == 12 or hour == 18):
            hotspot_data = day_data.copy()
        
        hotspot_data = pd.Series(data = np.ceil(hotspot_data/12), index = column_names)
        data = pd.concat([data, hotspot_data])
        train_hotspot_data.append(data)
    train_hotspot_data = pd.DataFrame(train_hotspot_data)

    # change resoution from daily to 6H (test) !!!always use the data from previous date
    test_daterange = pd.date_range(start = test_start_dt, end = test_end_dt, freq = '6H')
    test_hotspot_data = []
    for dt in test_daterange:
        current_date = dt.date()
        previous_date = dt.date() - timedelta(days = 1)
        hour = dt.time().hour
        
        data = pd.Series(data = [dt], index = ['date_time'])
        if (previous_date >= df_hotspot_test.iloc[0]['date']):
            night_data = df_hotspot_test.loc[df_hotspot_test["date"] == pd.to_datetime(str(previous_date), format = '%Y-%m-%d'), night_cols].iloc[0].to_numpy()
            day_data = df_hotspot_test.loc[df_hotspot_test["date"] == pd.to_datetime(str(previous_date), format = '%Y-%m-%d'), day_cols].iloc[0].to_numpy()
        
        else: #in case of the first date of the test intervals get the value of previous date from train data
            night_data = df_hotspot_train.loc[df_hotspot_train["date"] == pd.to_datetime(str(previous_date), format = '%Y-%m-%d'), night_cols].iloc[0].to_numpy()
            day_data = df_hotspot_train.loc[df_hotspot_train["date"] == pd.to_datetime(str(previous_date), format = '%Y-%m-%d'), day_cols].iloc[0].to_numpy()
    
        if (hour == 0 or hour == 6):
            hotspot_data = night_data.copy()
        elif (hour == 12 or hour == 18):
            hotspot_data = day_data.copy()
            
        hotspot_data = pd.Series(data = np.ceil(hotspot_data), index = column_names)
        data = pd.concat([data, hotspot_data])
        test_hotspot_data.append(data)
    test_hotspot_data = pd.DataFrame(test_hotspot_data)
    
    all_hotspot_data = pd.concat([train_hotspot_data, test_hotspot_data], axis = 0)
    return all_hotspot_data

In [5]:
#read hotspot scraped data
df_hotspot_scraped = read_scraped_data(base_scraped_data_path, "hotspot")
df_hotspot_scraped

,date,hotspot_thailand_day,hotspot_thailand_night,hotspot_myanmar_day,hotspot_myanmar_night,hotspot_cambodia_day,hotspot_cambodia_night,hotspot_lao_pdr_day,hotspot_lao_pdr_night,hotspot_p_malaysia_day,hotspot_p_malaysia_night
0,2017-07-01,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0
1,2017-07-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-07-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-07-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,2017-07-05,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1457,2021-06-27,6.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,2.0,0.0
1458,2021-06-28,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0
1459,2021-06-29,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0
1460,2021-06-30,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df_hotspot_processed = process_hotspot_data(df_hotspot_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, REGIONS)
#save processed hotspot data to csv files in ./processed_data/hotspot_processed_data.csv
save_result_from_processing(df_hotspot_processed, "hotspot")

df_hotspot_processed

,date_time,hotspot_thailand,hotspot_myanmar,hotspot_cambodia,hotspot_lao_pdr,hotspot_p_malaysia
0,2017-07-01 00:00:00,0.0,0.0,0.0,0.0,0.0
1,2017-07-01 06:00:00,0.0,0.0,0.0,0.0,0.0
2,2017-07-01 12:00:00,0.0,0.0,1.0,0.0,1.0
3,2017-07-01 18:00:00,0.0,0.0,1.0,0.0,1.0
4,2017-07-02 00:00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1459,2021-06-30 18:00:00,0.0,0.0,4.0,2.0,0.0
1460,2021-07-01 00:00:00,0.0,0.0,0.0,0.0,0.0
1461,2021-07-01 06:00:00,0.0,0.0,0.0,0.0,0.0
1462,2021-07-01 12:00:00,0.0,0.0,5.0,0.0,1.0


## Process Wind Data

1. Load data

   1.1) training wind data given in dataset
   
   1.2) scraped wind data 
   
2. Change both training's and scraped data's (testing) resolution to 1H

3. 3.1) In training data, we will perform interpolation
   - wind_dir (direction) --> perform cyclic interpolation, since direction ranges on [0, 360] degree
     (ex. 5, X, 355 --> X = 0) 
   - wind_speed --> perform simple linear interpolation
   
   3.2) In scraped wind data, which will further be used in test data. To avoid using future data, both wind_dir and wind_speed that are missing, the values will be substituted by its previous values only

4. sample back to 6H resolution

In [7]:
def perform_cyclic_interpolation(df):
    '''
    This function computes cyclic interpolation on wind direction data
    '''
    
    df = df.copy()
    df.columns.values[1] = 'wind_dir'
    
    df2 = df.loc[df['wind_dir'].notna(), ['date_time','wind_dir']].copy()
    df2.rename(columns = {'wind_dir': 'wind_dir_2'}, inplace = True)
    df2['wind_dir_2'] = np.rad2deg(np.unwrap(np.deg2rad(df2['wind_dir_2'])))
    
    df3 = pd.merge(df, df2, on = 'date_time', how = 'left')
    df3['wind_dir_2'] = df3['wind_dir_2'].interpolate().round(0)
    df3['wind_dir_2'] %= 360
    df3.drop(columns = 'wind_dir', inplace = True)
    df3.rename(columns = {'wind_dir_2': 'wind_dir'}, inplace = True)
    
    return df3['wind_dir']

def sample_df_freq(df, start_dt, end_dt, freq = '6h'):
    '''
    This function recieves a DataFrame change its resolution to freq given
    '''
    df = df.copy()
    df.set_index('date_time', inplace = True)
    df = df.reindex(pd.date_range(start_dt, end_dt, freq = freq), fill_value = np.nan)
    df.reset_index(inplace = True)
    df.rename(columns = {'index': 'date_time'}, inplace = True)
    return df
    
    
def process_wind_data(df_wind_train, df_wind_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, LOCS):
    # --> training data ------------------------------------------------------------------------
    # change resolution to 1H
    train_daterange_1H = pd.date_range(start = train_start_dt, end = train_end_dt, freq = 'H').to_frame(name = 'date_time').reset_index(drop = True)
    df_wind_train = pd.merge(train_daterange_1H, df_wind_train, on = 'date_time', how = 'outer')
    
    # perform cyclic interpolation on wind_dir columns / simple linear interpolation on wind_speed columns 
    for province in LOCS:
        df_wind_train[f'wind_dir_{province}'] = perform_cyclic_interpolation(df_wind_train[['date_time', f'wind_dir_{province}']])
        df_wind_train[f'wind_speed_{province}'] = df_wind_train[f'wind_speed_{province}'].interpolate().round(0)

    # change resolution to 6H
    df_wind_train = sample_df_freq(df_wind_train, start_dt = train_start_dt, end_dt = train_end_dt)
    
    # drop the first row (since there is no data)
    df_wind_train = df_wind_train.iloc[1:]
    df_wind_train.reset_index(inplace = True, drop = True)
    
    # --> testing data ------------------------------------------------------------------------
    # change resolution to 1H
    test_daterange_1H = pd.date_range(start = test_start_dt, end = test_end_dt, freq = 'H').to_frame(name = 'date_time').reset_index(drop = True)
    df_wind_scraped = pd.merge(test_daterange_1H, df_wind_scraped, on = 'date_time', how = 'outer')
    
    # for wind_speed and wind_dir in test data. To prevent using future data, we fill the missing values with
    # the previous values only
    for col in df_wind_scraped.columns[1:]:
        df_wind_scraped[col] = df_wind_scraped[col].interpolate(method = 'pad', limit_direction = 'forward')
    
    # for the first entry that has missing value, we fill by using the latest values from training data
    df_wind_scraped.iloc[0, 1:] = df_wind_train.iloc[-1, 1:]
    
    # change reoslution to 6H
    df_wind_scraped = sample_df_freq(df_wind_scraped, start_dt = test_start_dt, end_dt = test_end_dt)
    
    # -----------------------------------------------------------------------------------------
    all_wind_data = pd.concat([df_wind_train, df_wind_scraped], axis = 0)
    return all_wind_data

In [8]:
#load all wind training data -> concat 
df_wind_train = []
for i in range (len(LOCS)):
    province = LOCS[i]
    df = read_wind_or_temp_train_data(base_dataset_path, province, "wind")
    if (i != 0): #ignore date_time columns (except the first province)
        df = df.iloc[:, 1:]
    df_wind_train.append(df)
    
df_wind_train = pd.concat(df_wind_train, axis = 1)
df_wind_train

,date_time,wind_speed_BKK,wind_dir_BKK,wind_speed_Chiangmai,wind_dir_Chiangmai,wind_speed_Khonkaen,wind_dir_Khonkaen,wind_speed_Rayong,wind_dir_Rayong,wind_speed_Saraburi,wind_dir_Saraburi,wind_speed_Surat,wind_dir_Surat
0,2017-07-01 01:00:00,16,260,32,235,28,235,21,250,25,245,14,230
1,2017-07-01 04:00:00,16,255,30,250,29,245,21,250,25,250,14,235
2,2017-07-01 07:00:00,17,235,30,250,28,245,22,230,25,240,16,235
3,2017-07-01 10:00:00,18,240,26,245,29,245,24,235,26,245,17,240
4,2017-07-01 13:00:00,18,255,24,245,31,240,24,245,21,240,17,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8763,2020-06-30 10:00:00,24,250,18,265,18,255,36,255,35,255,18,235
8764,2020-06-30 13:00:00,23,250,12,230,22,230,32,255,25,240,15,230
8765,2020-06-30 16:00:00,37,255,12,235,21,240,41,265,27,255,19,255
8766,2020-06-30 19:00:00,36,265,16,235,20,235,36,280,31,245,19,250


In [9]:
# load wind scraped data
df_wind_scraped = read_scraped_data(base_scraped_data_path, "wind")
# rename columns
new_col_names = []
for col in df_wind_scraped.columns:
    if (col.startswith('wind_direction')):
        province = col.split('_')[2]
        new_col_names.append(f'wind_dir_{province}')
    else:
        new_col_names.append(col)
df_wind_scraped.columns = new_col_names


df_wind_scraped

,date_time,wind_speed_BKK,wind_dir_BKK,wind_speed_Chiangmai,wind_dir_Chiangmai,wind_speed_Khonkaen,wind_dir_Khonkaen,wind_speed_Rayong,wind_dir_Rayong,wind_speed_Saraburi,wind_dir_Saraburi,wind_speed_Surat,wind_dir_Surat
0,2020-06-30 22:00:00,35,270,17,225,21,230,47,270,41,250,23,260
1,2020-07-01 01:00:00,42,260,24,265,37,240,41,260,41,255,21,255
2,2020-07-01 04:00:00,55,270,23,280,39,255,46,260,49,265,29,255
3,2020-07-01 07:00:00,53,250,18,265,36,260,40,255,52,260,35,250
4,2020-07-01 10:00:00,51,260,22,265,34,265,38,250,61,260,36,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5831,2021-07-01 19:00:00,18,250,23,250,31,225,25,255,23,240,9,245
5832,2021-07-01 20:00:00,21,255,25,250,31,225,26,255,24,235,9,245
5833,2021-07-01 21:00:00,20,250,26,255,30,225,26,250,22,240,9,235
5834,2021-07-01 22:00:00,16,245,27,260,28,220,24,245,25,240,9,235


In [10]:
# bring wind train data and scraped wind data to process
df_wind_processed = process_wind_data(df_wind_train, df_wind_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, LOCS)
save_result_from_processing(df_wind_processed, "wind")

print(df_wind_processed.info())
df_wind_processed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5847 entries, 0 to 1463
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date_time             5847 non-null   datetime64[ns]
 1   wind_speed_BKK        5847 non-null   float64       
 2   wind_dir_BKK          5847 non-null   float64       
 3   wind_speed_Chiangmai  5847 non-null   float64       
 4   wind_dir_Chiangmai    5847 non-null   float64       
 5   wind_speed_Khonkaen   5847 non-null   float64       
 6   wind_dir_Khonkaen     5847 non-null   float64       
 7   wind_speed_Rayong     5847 non-null   float64       
 8   wind_dir_Rayong       5847 non-null   float64       
 9   wind_speed_Saraburi   5847 non-null   float64       
 10  wind_dir_Saraburi     5847 non-null   float64       
 11  wind_speed_Surat      5847 non-null   float64       
 12  wind_dir_Surat        5847 non-null   float64       
dtypes: datetime64[ns](

,date_time,wind_speed_BKK,wind_dir_BKK,wind_speed_Chiangmai,wind_dir_Chiangmai,wind_speed_Khonkaen,wind_dir_Khonkaen,wind_speed_Rayong,wind_dir_Rayong,wind_speed_Saraburi,wind_dir_Saraburi,wind_speed_Surat,wind_dir_Surat
0,2017-07-01 06:00:00,17.0,242.0,30.0,250.0,28.0,245.0,22.0,237.0,25.0,243.0,15.0,235.0
1,2017-07-01 12:00:00,18.0,250.0,25.0,245.0,30.0,242.0,24.0,242.0,23.0,242.0,17.0,247.0
2,2017-07-01 18:00:00,26.0,255.0,23.0,255.0,27.0,243.0,27.0,248.0,28.0,235.0,21.0,258.0
3,2017-07-02 00:00:00,20.0,262.0,22.0,250.0,30.0,248.0,24.0,248.0,27.0,255.0,18.0,238.0
4,2017-07-02 06:00:00,18.0,247.0,20.0,253.0,30.0,267.0,24.0,247.0,31.0,253.0,25.0,248.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,2021-06-30 18:00:00,29.0,270.0,19.0,250.0,30.0,235.0,26.0,250.0,27.0,260.0,14.0,270.0
1460,2021-07-01 00:00:00,20.0,265.0,28.0,270.0,36.0,235.0,23.0,255.0,30.0,255.0,11.0,255.0
1461,2021-07-01 06:00:00,21.0,275.0,19.0,265.0,25.0,255.0,18.0,260.0,24.0,265.0,11.0,255.0
1462,2021-07-01 12:00:00,17.0,245.0,19.0,250.0,21.0,230.0,17.0,240.0,24.0,230.0,9.0,235.0


## Process Temp Data

1. Load data

   1.1) training temp data given in dataset
   
   1.2) scraped temp data 
   
2. Change both training's and scraped data's (testing) resolution to 1H

3. 3.1) In training data, we will perform simple linear interpolation

   3.2) In scraped temp data, which will further be used in test data. To avoid using future data, the missing temp values will be substituted by its previous values only

4. sample back to 6H resolution

In [11]:
def process_temp_data(df_temp_train, df_temp_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, LOCS):
    # --> training data ------------------------------------------------------------------------
    # change resolution to 1H
    train_daterange_1H = pd.date_range(start = train_start_dt, end = train_end_dt, freq = 'H').to_frame(name = 'date_time').reset_index(drop = True)
    df_temp_train = pd.merge(train_daterange_1H, df_temp_train, on = 'date_time', how = 'outer')  
    
    # perform simple linear interpolation on temp columns
    for col in df_temp_train.columns[1:]:
        df_temp_train[col] = df_temp_train[col].interpolate().round(1)
        
    # change resolution to 6H
    df_temp_train = sample_df_freq(df_temp_train, start_dt = train_start_dt, end_dt = train_end_dt)
    
    # drop the first row (since there is no data)
    df_temp_train = df_temp_train.iloc[1:]
    df_temp_train.reset_index(inplace = True, drop = True)
    
    # --> testing data ------------------------------------------------------------------------
    # change resolution to 1H
    test_daterange_1H = pd.date_range(start = test_start_dt, end = test_end_dt, freq = 'H').to_frame(name = 'date_time').reset_index(drop = True)
    df_temp_scraped = pd.merge(test_daterange_1H, df_temp_scraped, on = 'date_time', how = 'outer')
    
    # to prevent using future data, fill the missing values with only its previous values (in test data)
    for col in df_temp_scraped.columns[1:]:
        df_temp_scraped[col] = df_temp_scraped[col].interpolate(method = 'pad', limit_direction = 'forward')
    
    # for the first entry that has missing value, we fill by using the latest values from training data
    df_temp_scraped.iloc[0, 1:] = df_temp_train.iloc[-1, 1:]
    
    # change resolution to 6H
    df_temp_scraped = sample_df_freq(df_temp_scraped, start_dt = test_start_dt, end_dt = test_end_dt)
    # -----------------------------------------------------------------------------------------
    
    all_temp_data = pd.concat([df_temp_train, df_temp_scraped], axis = 0)
    return all_temp_data

In [12]:
# load all train wind data -> concat
df_temp_train = []
for i in range (len(LOCS)):
    province = LOCS[i]
    df = read_wind_or_temp_train_data(base_dataset_path, province, "temp")
    if (i != 0): #ignore date_time columns (except the first province)
        df = df.iloc[:, 1:]
    df_temp_train.append(df)
    
df_temp_train = pd.concat(df_temp_train, axis = 1)
df_temp_train

,date_time,temp_BKK,temp_Chiangmai,temp_Khonkaen,temp_Rayong,temp_Saraburi,temp_Surat
0,2017-07-01 01:00:00,27.9,23.2,26.5,27.9,26.6,24.6
1,2017-07-01 04:00:00,27.4,22.3,25.8,27.7,26.0,23.9
2,2017-07-01 07:00:00,28.0,24.3,26.5,27.9,26.9,25.8
3,2017-07-01 10:00:00,31.1,29.3,29.5,29.4,31.6,29.9
4,2017-07-01 13:00:00,33.3,30.8,31.8,29.9,34.2,31.7
...,...,...,...,...,...,...,...
8763,2020-06-30 10:00:00,30.8,28.9,29.6,28.7,30.3,28.7
8764,2020-06-30 13:00:00,32.0,32.2,29.6,28.7,32.9,31.6
8765,2020-06-30 16:00:00,32.1,32.2,29.6,28.0,31.8,30.8
8766,2020-06-30 19:00:00,30.3,27.5,29.6,27.9,27.4,26.7


In [13]:
# load temp scraped data
df_temp_scraped = read_scraped_data(base_scraped_data_path, "temp")
df_temp_scraped

,date_time,temp_BKK,temp_Chiangmai,temp_Khonkaen,temp_Rayong,temp_Saraburi,temp_Surat
0,2020-06-30 22:00:00,29.4,25.2,27.6,28.5,25.9,25.9
1,2020-07-01 01:00:00,28.5,24.0,26.7,28.2,25.5,25.2
2,2020-07-01 04:00:00,28.3,23.4,25.5,27.8,25.1,24.7
3,2020-07-01 07:00:00,28.7,24.1,26.0,28.2,25.8,25.4
4,2020-07-01 10:00:00,29.7,29.7,29.4,29.6,28.7,29.3
...,...,...,...,...,...,...,...
5831,2021-07-01 19:00:00,29.1,24.6,30.7,28.5,29.7,26.8
5832,2021-07-01 20:00:00,29.5,24.1,29.8,28.3,28.0,26.2
5833,2021-07-01 21:00:00,29.8,23.9,28.9,28.2,27.0,25.9
5834,2021-07-01 22:00:00,29.1,23.5,28.0,28.2,26.0,25.7


In [14]:
# bring temp train data and scraped temp data to process
df_temp_processed = process_temp_data(df_temp_train, df_temp_scraped, train_start_dt, train_end_dt, test_start_dt, test_end_dt, LOCS)
save_result_from_processing(df_temp_processed, "temp")

print(df_temp_processed.info())
df_temp_processed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5847 entries, 0 to 1463
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date_time       5847 non-null   datetime64[ns]
 1   temp_BKK        5847 non-null   float64       
 2   temp_Chiangmai  5847 non-null   float64       
 3   temp_Khonkaen   5847 non-null   float64       
 4   temp_Rayong     5847 non-null   float64       
 5   temp_Saraburi   5847 non-null   float64       
 6   temp_Surat      5847 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 365.4 KB
None


,date_time,temp_BKK,temp_Chiangmai,temp_Khonkaen,temp_Rayong,temp_Saraburi,temp_Surat
0,2017-07-01 06:00:00,27.8,23.6,26.3,27.8,26.6,25.2
1,2017-07-01 12:00:00,32.6,30.3,31.0,29.7,33.3,31.1
2,2017-07-01 18:00:00,30.1,27.6,30.2,28.6,30.6,27.9
3,2017-07-02 00:00:00,28.1,23.6,26.8,27.6,26.5,24.9
4,2017-07-02 06:00:00,27.4,24.0,26.0,27.4,26.2,25.0
...,...,...,...,...,...,...,...
1459,2021-06-30 18:00:00,26.0,28.1,31.6,29.1,30.2,28.7
1460,2021-07-01 00:00:00,28.4,23.2,26.7,28.2,25.4,24.9
1461,2021-07-01 06:00:00,26.5,21.9,24.9,27.7,24.4,23.8
1462,2021-07-01 12:00:00,32.3,28.5,34.3,30.2,33.5,32.2
